---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [86]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)
print(df[39:41])

39    1-14-81 Communication with referring physician...
40    7-29-75 CPT Code: 90801 - Psychiatric Diagnosi...
dtype: object


In [118]:
import numpy as np
from datetime import datetime

def date_sorter():
   
     # Your code here

    data = pd.DataFrame(data=df)

    dates_extracted = df.str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4}))')
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2}))'))
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    del dates_extracted[3]
    del dates_extracted[4]
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4}))'))
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4}))'))
    del dates_extracted[3]
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Without day
    dates_without_day = df[index_left].str.extractall('(?P<origin>(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4}))')
    dates_without_day = dates_without_day.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d\d?)/(?P<year>\d{4}))'))
    dates_without_day['day'] = 1
    dates_extracted = dates_extracted.append(dates_without_day)
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Only year
    dates_only_year = df[index_left].str.extractall(r'(?P<origin>(?P<year>\d{4}))')
    dates_only_year['day'] = 1
    dates_only_year['month'] = 1
    dates_extracted = dates_extracted.append(dates_only_year)
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Year
    dates_extracted['year'] = dates_extracted['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    dates_extracted['year'] = dates_extracted['year'].apply(lambda x: str(x))

    # Month
    dates_extracted['month'] = dates_extracted['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
    dates_extracted.replace({"month": month_dict}, inplace=True)
    dates_extracted['month'] = dates_extracted['month'].apply(lambda x: str(x))

    # Day
    dates_extracted['day'] = dates_extracted['day'].apply(lambda x: str(x))

    # Cleaned date
    dates_extracted['date'] = dates_extracted['month'] + '/' + dates_extracted['day'] + '/' + dates_extracted['year']
    dates_extracted['date'] = pd.to_datetime(dates_extracted['date'])

    dates_extracted.sort_values(by='date', inplace=True)
    df1 = pd.Series(list(dates_extracted.index.labels[0]))
    
    return df1

    '''
    Code	Meaning	Example
    %A	Weekday as locale’s full name.	Wednesday
    %a	Weekday as locale’s abbreviated name.	Wed
    %B	Month as locale’s full name.	June
    %d	Day of the month.	06
    %m	Month as a number.	6
    %Y	Four-digit year.	2018
    %y	Two-digit year.	18
    '''
    #04/20/2009; 04/20/09; 4/20/09; 4/3/09
    #Mar-20-2009;
    #Mar 20, 2009; March 20, 2009;
    #Mar. 20, 2009;
    #Mar 20 2009;
    #20 Mar 2009;
    #20 March 2009;
    #20 Mar. 2009;
    #20 March, 2009
    #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    #Feb 2009; Sep 2009; Oct 2010  
    #6/2008; 12/2009
    #2009; 2010
'''
    df1 =data.T.iloc[0].str.extract(r'((\d?\d)/(\d?\d)/(\d\d\d\d))')
    df1[0] = pd.to_datetime(df1[0].astype(str),format='%m/%d/%Y')
    
    df31 =data.T.iloc[0].str.extract(r'((\d?\d)-(\d?\d)-(\d\d\d\d))')
    df31[0] = pd.to_datetime(df31[0].astype(str),format='%m-%d-%Y')
    
    df32 =data.T.iloc[0].str.extract(r'((\d?\d)-(\d?\d)-(\d\d))')
    df32[0] = pd.to_datetime(df32[0].astype(str),format='%m-%d-%y')
    
    df30 =data.T.iloc[0].str.extract(r'(([12]{1,1}\d\d\d))')
   
    df30[0] = pd.to_datetime(df30[0].astype(str),format='%Y')

    #Mar-20-2009
    df19 =data.T.iloc[0].str.extract(r'((?:January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|October|Nov|Dec)-(\d?\d)-(\d\d\d\d))')
    df19[0] = pd.to_datetime(df19[0].astype(str),format='%b-%d-%Y')
    #print(df19.dropna())
    df20 =data.T.iloc[0].str.extract(r'((?:January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|October|Nov|Dec)-(\d?\d)-(\d\d\d\d))')
    df20[0] = pd.to_datetime(df20[0].astype(str),format='%b-%d-%y')
    #print(df20.dropna())
    df21 =data.T.iloc[0].str.extract(r'((?:January|February|March|April|May|June|July|August|September|October|November|December) (\d?\d), (\d\d\d\d))')
    df21[0] = pd.to_datetime(df21[0].astype(str),format='%B %d, %Y')
    #print(df21.dropna())
    df23 =data.T.iloc[0].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|October|Nov|Dec) (\d?\d), (\d\d\d\d))')
    df23[0] = pd.to_datetime(df23[0].astype(str),format='%b %d, %Y')
    #print(df21.dropna())
    df22 =data.T.iloc[0].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.] (\d?\d), (\d\d\d\d))')
    df22[0] = pd.to_datetime(df22[0].astype(str),format='%b. %d, %Y')
    #print(df22.dropna())
    #20 Mar. 2009;
    df24 =data.T.iloc[0].str.extract(r'((\d?\d) (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.] (\d\d\d\d))')
    df24[0] = pd.to_datetime(df24[0].astype(str),format='%d %b. %Y')
    #print(df24.dropna())
     #20 March, 2009
    df25 =data.T.iloc[0].str.extract(r'((\d?\d) (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[,] (\d\d\d\d))')
    df25[0] = pd.to_datetime(df25[0].astype(str),format='%d %b, %Y')
    #print(df25.dropna())
    df26 =data.T.iloc[0].str.extract(r'((\d?\d) (?:January|February|March|April|May|June|July|August|September|October|November|December), (\d\d\d\d))')
    df26[0] = pd.to_datetime(df26[0].astype(str),format='%d %B, %Y')
    #print(df26.dropna())
    ##Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    df27 =data.T.iloc[0].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (\d?\d)th, (\d\d\d\d))')
    df27[0] = pd.to_datetime(df27[0].astype(str),format='%B %dth, %Y')
    #print(df27.dropna())
    df28 =data.T.iloc[0].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (\d?\d)st, (\d\d\d\d))')
    df28[0] = pd.to_datetime(df28[0].astype(str),format='%B %dst, %Y')
    #print(df28.dropna())
    df29 =data.T.iloc[0].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (\d?\d)nd, (\d\d\d\d))')
    df29[0] = pd.to_datetime(df29[0].astype(str),format='%B %dnd, %Y')
    #print(df29.dropna())
    
    df6 =data.T.iloc[0].str.extract(r'((\d?\d)/(\d?\d)/(\d\d))')
    df6[0] = pd.to_datetime(df6[0].astype(str),format='%m/%d/%y')
    #print(df6.dropna())
    
    df2 =data.T.iloc[0].str.extract(r'((\d?\d)/(\d\d))')  
    #print(df2[0])
    df2[0] = pd.to_datetime(df2[0].astype(str),format='%m/%y')
    
    
    df7 =data.T.iloc[0].str.extract(r'([0]?(?:1|2|3|4|5|6|7|8|9|10|11|12)/(\d\d\d\d))')  
    df7[0] = pd.to_datetime(df7[0].astype(str),format='%m/%Y')
    #print(df7.dropna())
    
    df3 = data.T.iloc[0].str.extract(r'([12]\d\d\d)')#r'(\w\s([12]\d\d\d))'
    #print(df3)
    #df3 = df3.T.iloc[0].str.replace(r'\w\s','1/1/')
    df3 = pd.to_datetime(df3.astype(str),format='%Y')#%m/%d/%Y
    #print(df3.dropna())
        
    ##df4 = data.T.iloc[0].str.extract(r'([\d]{0,2}(\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)[.,]{0,1}\s([12]\d\d\d))')
    ##df4 = data.T.iloc[0].str.extract(r'(\b\d{1,2}\D{0,3})?\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.\-\/])\D?)?((19[7-9]\d|20\d{2})|\d{2})*')
    ##df5 = data.T.iloc[0].str.extract(r'((\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)[-]{0,1} [\d]{0,2}[th|nd|st]{0,1}[-]{0,1} [.,]{0,1}\s([12]\d\d\d))')
    
    #df4 = data.T.iloc[0].str.extract(r'(\b[\d]{0,2} (?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)[.,]{0,1}\s([12]\d\d\d))')  
    #print(df4.dropna())
    #df4 = datetime.strptime(date_str1, '%d, %b, %Y')
  
    df5 = data.T.iloc[0].str.extract(r'(\b[\d]{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.,]{0,1}\s([12]\d\d\d))')  
    #print(df41[0].dropna())
    df5[0] = pd.to_datetime(df5[0].astype(str), format='%d %b %Y')
    #print(df41.dropna())
    df11 = data.T.iloc[0].str.extract(r'(\b[\d]{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.,]{0,1}\s(\d\d))')  
    #print(df11.dropna())
    df11[0] = pd.to_datetime(df11[0].astype(str), format='%d %b %y')
    #print(df11.dropna())
    

    df6 = data.T.iloc[0].str.extract(r'(\b[\d]{1,2} (?:January|February|March|April|May|June|July|August|September|October|November|December)[.,]{0,1}\s([12]\d\d\d))')   
    df6[0] = pd.to_datetime(df6[0].astype(str), format='%d %B %Y')
    #print(df411.dropna())
    df12 = data.T.iloc[0].str.extract(r'(\b[\d]{1,2} (?:January|February|March|April|May|June|July|August|September|October|November|December)[.,]{0,1}\s(\d\d))')   
    df12[0] = pd.to_datetime(df12[0].astype(str), format='%d %B %y')
    #print(df12.dropna())
    
    df7 = data.T.iloc[0].str.extract(r'(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.,]\s([12]\d\d\d))')  
    #print(df42.dropna())
    df7[0] = pd.to_datetime(df7[0].astype(str), format='%b, %Y')
    #print(df42.dropna())
    df13 = data.T.iloc[0].str.extract(r'(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[,]\s(\d\d))')  
    df13[0] = pd.to_datetime(df13[0].astype(str), format='%b, %y')
    #print(df13.dropna())
    
    df14 = data.T.iloc[0].str.extract(r'(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.]\s(\d\d))')  
    df14[0] = pd.to_datetime(df14[0].astype(str), format='%b. %y')
    #print(df14.dropna())
    
    df8 = data.T.iloc[0].str.extract(r'(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s([12]\d\d\d))')  
    #print(df42.dropna())
    df8[0] = pd.to_datetime(df8[0].astype(str), format='%b %Y')
    #print(df420.dropna())
    df15 = data.T.iloc[0].str.extract(r'(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d\d))')  
    #print(df42.dropna())
    df15[0] = pd.to_datetime(df15[0].astype(str), format='%b %y')
    #print(df15.dropna())
    
    df9 = data.T.iloc[0].str.extract(r'(\b (?:January|February|March|April|May|June|July|August|September|October|November|December)[.,]\s([12]\d\d\d))')  
    df9[0] = pd.to_datetime(df9[0].astype(str), format=' %B, %Y')
    #print(df421.dropna())
   # df16 = data.T.iloc[0].str.extract(r'(\b (?:January|February|March|April|May|June|July|August|September|October|November|December)[.]\s(\d\d))')  
   # df16[0] = pd.to_datetime(df16[0].astype(str), format=' %B. %y')
    #print(df16.dropna())
    df17 = data.T.iloc[0].str.extract(r'(\b (?:January|February|March|April|May|June|July|August|September|October|November|December)[,]\s(\d\d))')  
    df17[0] = pd.to_datetime(df17[0].astype(str), format=' %B, %y')
    #print(df17.dropna())
    

    df10 = data.T.iloc[0].str.extract(r'(\b (?:January|February|March|April|May|June|July|August|September|October|November|December)\s([12]\d\d\d))')  
    df10[0] = pd.to_datetime(df10[0].astype(str), format=' %B %Y')
    #print(df4211.dropna())
    df18 = data.T.iloc[0].str.extract(r'(\b (?:January|February|March|April|May|June|July|August|September|October|November|December)\s(\d\d))')  
    df18[0] = pd.to_datetime(df18[0].astype(str), format=' %B %y')
    
    #df5 = data.T.iloc[0].str.extract(r' (\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)[-]{0,1} [\d]{0,2}[th|nd|st]{0,1}[-]{0,1} [.,]{0,1}\s([12]\d\d\d))')
    #print(df5.dropna())
        
    #print(type(df1))
    result = pd.DataFrame(data =df1.T.iloc[0].dropna()).append(pd.DataFrame(data=df6[0].dropna()))
    result = result.append(pd.DataFrame(data=df2[0].dropna()))
    result = result.append(pd.DataFrame(data=df7[0].dropna()))
    result = result.append(pd.DataFrame(data=df3.dropna()))
    #result = result.append(pd.DataFrame(data=df4.dropna()))
    result = result.append(pd.DataFrame(data=df5[0].dropna()))
    result = result.append(pd.DataFrame(data=df6[0].dropna()))
    result = result.append(pd.DataFrame(data=df7[0].dropna()))
    result = result.append(pd.DataFrame(data=df8[0].dropna()))
    result = result.append(pd.DataFrame(data=df9[0].dropna()))
    result = result.append(pd.DataFrame(data=df10[0].dropna()))
    result = result.append(pd.DataFrame(data=df11[0].dropna()))
    result = result.append(pd.DataFrame(data=df12[0].dropna()))
    result = result.append(pd.DataFrame(data=df13[0].dropna()))
    result = result.append(pd.DataFrame(data=df14[0].dropna()))
    result = result.append(pd.DataFrame(data=df15[0].dropna()))
   # result = result.append(pd.DataFrame(data=df16[0].dropna()))
    result = result.append(pd.DataFrame(data=df17[0].dropna()))
    result = result.append(pd.DataFrame(data=df18[0].dropna()))
    result = result.append(pd.DataFrame(data=df19[0].dropna()))
    result = result.append(pd.DataFrame(data=df20[0].dropna()))
    result = result.append(pd.DataFrame(data=df21[0].dropna()))
    result = result.append(pd.DataFrame(data=df22[0].dropna()))
    result = result.append(pd.DataFrame(data=df23[0].dropna()))
    result = result.append(pd.DataFrame(data=df24[0].dropna()))
    result = result.append(pd.DataFrame(data=df25[0].dropna()))
    result = result.append(pd.DataFrame(data=df26[0].dropna()))
    result = result.append(pd.DataFrame(data=df27[0].dropna()))
    result = result.append(pd.DataFrame(data=df28[0].dropna()))
    result = result.append(pd.DataFrame(data=df29[0].dropna()))
    result = result.append(pd.DataFrame(data=df30[0].dropna()))
    result = result.append(pd.DataFrame(data=df31[0].dropna()))
    result = result.append(pd.DataFrame(data=df32[0].dropna()))
   
    
    result.sort_index(inplace=True)
    result = result.loc[~result.index.duplicated(keep='first')]
   

    result.sort_values(by=0, inplace=True)
    df1 = pd.Series(list(result.index))
    
    #result =result.set_index(0)    
    #result['index']= range(len(result))
    #result.sort_index(inplace=True)  
    #result['line']=range(len(result))
    #result=result.set_index('line')
    
   
    return df1#result['index']
'''
    
   

"\n    df1 =data.T.iloc[0].str.extract(r'((\\d?\\d)/(\\d?\\d)/(\\d\\d\\d\\d))')\n    df1[0] = pd.to_datetime(df1[0].astype(str),format='%m/%d/%Y')\n    \n    df31 =data.T.iloc[0].str.extract(r'((\\d?\\d)-(\\d?\\d)-(\\d\\d\\d\\d))')\n    df31[0] = pd.to_datetime(df31[0].astype(str),format='%m-%d-%Y')\n    \n    df32 =data.T.iloc[0].str.extract(r'((\\d?\\d)-(\\d?\\d)-(\\d\\d))')\n    df32[0] = pd.to_datetime(df32[0].astype(str),format='%m-%d-%y')\n    \n    df30 =data.T.iloc[0].str.extract(r'(([12]{1,1}\\d\\d\\d))')\n   \n    df30[0] = pd.to_datetime(df30[0].astype(str),format='%Y')\n\n    #Mar-20-2009\n    df19 =data.T.iloc[0].str.extract(r'((?:January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|October|Nov|Dec)-(\\d?\\d)-(\\d\\d\\d\\d))')\n    df19[0] = pd.to_datetime(df19[0].astype(str),format='%b-%d-%Y')\n    #print(df19.dropna())\n    df20 =data.T.iloc[0].str.extract(r'((?:January|February|March|April|May|June|Jul

In [119]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    286
477    244
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64